## 상품정보 데이터

### json데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

import warnings
import os
warnings.filterwarnings(action='ignore')
#pd.set_option('옵션 이름', 수정할 값)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [28]:
items = pd.read_json('./items.json')
items.head()



,cat,name,img,price,link
0,DRESS,[오바오원피스🐼][이너나시원피스set][날씬해보여][MADE] 프롬스 윙소매 셔츠미...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,29800,https://hotping.co.kr/product/%EC%98%A4%EB%B0%...
1,DRESS,[썸머버전추가💙][MADE] 르본 링클지 브이넥 반팔 롱원피스\n(44~100) (...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,36090,https://hotping.co.kr/product/%EC%8D%B8%EB%A8%...
2,DRESS,[구김X/쾌적🌀][MADE] 위아크 브이넥 링클 반팔 롱원피스\n(44~110) (...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,32370,https://hotping.co.kr/product/%EA%B5%AC%EA%B9%...
3,DRESS,[활용도200%/통기성↑/시원&쾌적][MADE] 아틀리에 허리스트링 링클 레이어드 ...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,31440,https://hotping.co.kr/product/%ED%99%9C%EC%9A%...
4,DRESS,[1분코디/여름원피스☀️][MADE] 부티에 배색 스티치 반팔 롱원피스\n(44~1...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,34230,https://hotping.co.kr/product/1%EB%B6%84%EC%BD...


In [29]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cat     1023 non-null   object
 1   name    1023 non-null   object
 2   img     1023 non-null   object
 3   price   1023 non-null   int64 
 4   link    1023 non-null   object
dtypes: int64(1), object(4)
memory usage: 40.1+ KB


### 상품명 전처리

In [30]:
def item_name(row):
    return row.split('\n')[0] # 줄바꿈이전 문자열만 남기기

items['name'] = items['name'].apply(item_name)
items.head()

,cat,name,img,price,link
0,DRESS,[오바오원피스🐼][이너나시원피스set][날씬해보여][MADE] 프롬스 윙소매 셔츠미...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,29800,https://hotping.co.kr/product/%EC%98%A4%EB%B0%...
1,DRESS,[썸머버전추가💙][MADE] 르본 링클지 브이넥 반팔 롱원피스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,36090,https://hotping.co.kr/product/%EC%8D%B8%EB%A8%...
2,DRESS,[구김X/쾌적🌀][MADE] 위아크 브이넥 링클 반팔 롱원피스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,32370,https://hotping.co.kr/product/%EA%B5%AC%EA%B9%...
3,DRESS,[활용도200%/통기성↑/시원&쾌적][MADE] 아틀리에 허리스트링 링클 레이어드 ...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,31440,https://hotping.co.kr/product/%ED%99%9C%EC%9A%...
4,DRESS,[1분코디/여름원피스☀️][MADE] 부티에 배색 스티치 반팔 롱원피스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,34230,https://hotping.co.kr/product/1%EB%B6%84%EC%BD...


### 상품명 중복확인

In [31]:
dup_idx = items.loc[items['name'].duplicated()==True].index
items.drop(dup_idx, axis=0, inplace=True)
items['name'].duplicated().sum()


0

In [32]:
# 인덱스 재설정
items.reset_index(drop=True, inplace=True)

In [33]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cat     860 non-null    object
 1   name    860 non-null    object
 2   img     860 non-null    object
 3   price   860 non-null    int64 
 4   link    860 non-null    object
dtypes: int64(1), object(4)
memory usage: 33.7+ KB


### Item_id 부여

In [34]:
items['item_id'] = [(5-len((str(i)))) * '0' + str(i) for i in list(items.index)]
items.head()

,cat,name,img,price,link,item_id
0,DRESS,[오바오원피스🐼][이너나시원피스set][날씬해보여][MADE] 프롬스 윙소매 셔츠미...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,29800,https://hotping.co.kr/product/%EC%98%A4%EB%B0%...,00000
1,DRESS,[썸머버전추가💙][MADE] 르본 링클지 브이넥 반팔 롱원피스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,36090,https://hotping.co.kr/product/%EC%8D%B8%EB%A8%...,00001
2,DRESS,[구김X/쾌적🌀][MADE] 위아크 브이넥 링클 반팔 롱원피스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,32370,https://hotping.co.kr/product/%EA%B5%AC%EA%B9%...,00002
3,DRESS,[활용도200%/통기성↑/시원&쾌적][MADE] 아틀리에 허리스트링 링클 레이어드 ...,https://cafe24img.poxo.com/sseoqkr7/web/produc...,31440,https://hotping.co.kr/product/%ED%99%9C%EC%9A%...,00003
4,DRESS,[1분코디/여름원피스☀️][MADE] 부티에 배색 스티치 반팔 롱원피스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,34230,https://hotping.co.kr/product/1%EB%B6%84%EC%BD...,00004


In [35]:
items.tail()

,cat,name,img,price,link,item_id
855,BLOUSE / SHIRT,[MADE] 레일라 여름 시스루 퍼프 반팔블라우스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,9000,https://hotping.co.kr/product/made-%EB%A0%88%E...,00855
856,BLOUSE / SHIRT,[MADE] 노딩 배색라인 반팔블라우스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,12000,https://hotping.co.kr/product/made-%EB%85%B8%E...,00856
857,BLOUSE / SHIRT,[MADE][LOVB] L-999 하트넥 퍼프블라우스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,14000,https://hotping.co.kr/product/madelovb-l-999-%...,00857
858,BLOUSE / SHIRT,[여름원단/구김ZERO♥][MADE] 메르에 링클 주름 배색 레이어드 반팔 셔츠블라우스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,14000,https://hotping.co.kr/product/%EC%97%AC%EB%A6%...,00858
859,BLOUSE / SHIRT,[MADE] 미에르 러플 퍼프랩블라우스,https://cafe24img.poxo.com/sseoqkr7/web/produc...,15000,https://hotping.co.kr/product/made-%EB%AF%B8%E...,00859


In [36]:
items.to_json('../Load/items_prc.json', orient='records', indent=2, force_ascii=False) # 객체기준 레코드, 들여쓰기 적용, 한글깨짐 해결

In [37]:
# 리뷰 크롤링용 데이터 저장
items[['item_id', 'link']].to_json('../Extract/items_link.json', orient='records', indent=2, force_ascii=False)

In [48]:
review = pd.read_json('./reviews.json')

# 불필요한 컬럼 삭제
review = review[['품번', '아이디', '별점', '키', '몸무게', '평소사이즈', '타입', '리뷰', '사이즈', '컬러', '색상', '색상-사이즈', '타입2', '데님']]
review.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74473 entries, 0 to 74472
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      74473 non-null  int64 
 1   아이디     74473 non-null  object
 2   별점      74473 non-null  int64 
 3   키       57347 non-null  object
 4   몸무게     52402 non-null  object
 5   평소사이즈   53856 non-null  object
 6   타입      71965 non-null  object
 7   리뷰      74473 non-null  object
 8   사이즈     547 non-null    object
 9   컬러      388 non-null    object
 10  색상      2995 non-null   object
 11  색상-사이즈  286 non-null    object
 12  타입2     47 non-null     object
 13  데님      820 non-null    object
dtypes: int64(2), object(12)
memory usage: 8.0+ MB


* ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님'] 컬럼 통일 필요 = **['사이즈', '색상']으로 통일**

* 나머지 컬럼은 삭제

* 문자열 변수들 카디널리티 확인

* '별점' 컬럼 핫핑 직원 데이터는 제외

In [32]:
print('타입: ', review.loc[review['타입'].notnull()==True].iloc[0]['타입'])
print('사이즈: ', review.loc[review['사이즈'].notnull()==True].iloc[0]['사이즈'])
print('컬러: ', review.loc[review['컬러'].notnull()==True].iloc[0]['컬러'])
print('색상: ', review.loc[review['색상'].notnull()==True].iloc[0]['색상'])
print('색상-사이즈: ', review.loc[review['색상-사이즈'].notnull()==True].iloc[0]['색상-사이즈'])
print('데님: ', review.loc[review['데님'].notnull()==True].iloc[0]['데님'])

타입:  블랙(black)-3
사이즈:  2
컬러:  블랙
색상:  소라
색상-사이즈:  카키(khaki)-3
데님:  연청(denim)-2


In [49]:
review['타입'].unique()

array([None, '블랙(black)-3', '소라(light teal)-3', '블랙(black)-2',
       '소라(light teal)-2', '소라(light teal)-1', '블랙(black)-4',
       '블랙(black)-1', '소라(light teal)-4', '썸머_블랙(black)-1',
       '썸머_블랙(black)-3', '소라(sora)-2', '소라(sora)-1', '소라(sora)-3',
       '브라운(brown)-1', '브라운(brown)-3', '진베이지(D.beige)-3', '네이비(navy)-2',
       '네이비(navy)-3', '블루(blue)-2', '라즈베리핑크(R.pink)-3',
       '라즈베리핑크(R.pink)-1', '딥네이비(D.navy)-3', '딥네이비(D.navy)-1',
       '딥네이비(D.navy)-2', '클래식블루(C.blue)-2', '클래식블루(C.blue)-1',
       '클래식블루(C.blue)-3', '화이트(white)-2', '화이트(white)-3', '카키(khaki)-2',
       '화이트(white)-1', '카키(khaki)-1', '카키(khaki)-3', '네이비(navy)-1',
       '연그레이(L.grey)-2', '옐로우(yellow)-2', '연그레이(L.grey)-1',
       '옐로우(yellow)-1', '크림(cream)-3', '크림(cream)-1', '연카키(L.khaki)-2',
       '라이트옐로우(L.yellow)-2', '라이트옐로우(L.yellow)-1', '라이트옐로우(L.yellow)-3',
       '연청(denim)-2', '진청(D.denim)-1', '데님(denim)-1', '데님(denim)-2',
       '진청(D.denim)-2', '연청(denim)-1', '진청(D.denim)-3', '연청(denim)-3',
       

In [52]:
review['타입'].notnull().sum()

71965

In [ ]:
for idx, row in review.itertuples(name=None):
    

In [54]:
review.loc[(review['타입'].notnull()==True) & (review['색상'].notnull()==True)]

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,컬러,색상,색상-사이즈,타입2,데님
10979,41,4832981,5,156~160,51 ~ 55,55,후드,완전 귀여워요. 핫핑에서 이번에 원피스 4벌샀는데 이게 쩰 맘에듬\n157/54 1...,NaN,NaN,진베이지(D.beige)-F,NaN,NaN,NaN
68772,360,핫핑-다슬,5,156~160,46 ~ 50,55,브이,미친가격에 고퀄리티의 트위드자켓 이라니...✨😎\n말 그대로 가성비 갑 아이템이에요...,F,NaN,옐로우,NaN,NaN,NaN
71584,418,181147926,5,161~165,65~,77,12.찰리레터링,기본 검정티인데 찰리레터링이라 포인트되고 좋은거 같아요! 무난하게 입기 좋아요~,NaN,NaN,블랙(black),NaN,NaN,NaN
71585,418,2794172,5,161~165,65~,77,11.스페이스,"스누피가 그려져 있어서 좋고, 재질도 두꺼워서 잘 비치지 않습니다.\n오버핏이라 너...",NaN,NaN,화이트(white),NaN,NaN,NaN
71586,418,lovewk,5,166~170,None,66,17.디너,작년에 사고 이번에 다른색으로 또 샀어요. 마음에 들어요\n길이도 길고 박시하게 이쁘네요,NaN,NaN,블루(blue),NaN,NaN,NaN
71587,418,2890976,5,161~165,61 ~ 65,77,46.파리스,검정 바지랑 입을려고 주문했어요.\n예뻐요.\n프린팅 아니고 자수라서 더 고급스러움^^,NaN,NaN,아이보리(ivory),NaN,NaN,NaN
71588,418,11929525,5,None,None,None,12.찰리레터링,전부터 여름옷은 여기에서 사는데 다 맘에 들어요~옷 색이 맘에 들고 길이도 적당해요...,NaN,NaN,화이트(white),NaN,NaN,NaN
71589,418,229080055,5,161~165,56 ~ 60,66,02.하트러브헐,66반에 165인데 사이즈 넉넉하구여 그렇게 많이 얇지 얂아서 좋구 잘입구 다니구 ...,NaN,NaN,화이트(white),NaN,NaN,NaN
71590,418,89732980,5,161~165,65~,77,40.R베어,작게 있는 곰로고모양이 포인트로 넘 귀여워요-^^ 많이 비치지도 않고 넘 두껍지도 ...,NaN,NaN,화이트(white),NaN,NaN,NaN
71591,418,233347906,5,161~165,51 ~ 55,66,41.파스타,넉넉하니 편하니 핏도 예쁘고 너무 좋아요~\n여름에 찾던 스타일이에요~,NaN,NaN,화이트(white),NaN,NaN,NaN


In [ ]:
def prc_type(row):
    
    # 사이즈-색상
    # 사이즈-컬러
    # 색상-사이즈
    # 데님, 타입2, 타입
    
